<a href="https://colab.research.google.com/github/bzhao10/cs598_dlh_final_project/blob/main/models_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
notes=pd.read_csv('/content/drive/MyDrive/DLH_final_project_dataset/preprocessed_reg_icd_codes_10000.csv')
notes.columns

Index(['Unnamed: 0', 'TEXT', 'reg_code_00845', 'reg_code_0088',
       'reg_code_0090', 'reg_code_0091', 'reg_code_01085', 'reg_code_0311',
       'reg_code_0380', 'reg_code_03811',
       ...
       'category_Case Management', 'category_Consult', 'category_General',
       'category_Nursing/other', 'category_Nutrition', 'category_Pharmacy',
       'category_Physician', 'category_Rehab Services', 'category_Respiratory',
       'category_Social Work'],
      dtype='object', length=1166)

In [ ]:
reg_codes_list = notes.loc[:, notes.columns.str.startswith('reg_code_')].columns.tolist()
print(len(reg_codes_list))


1154


In [ ]:
rolled_up_codes_list=['rolled_up_code_518','rolled_up_code_427','rolled_up_code_584','rolled_up_code_428','rolled_up_code_276','rolled_up_code_401','rolled_up_code_285','rolled_up_code_414','rolled_up_code_272','rolled_up_code_038']

In [ ]:
pd.value_counts(notes.rolled_up_code_518)

1    5575
0    4425
Name: rolled_up_code_518, dtype: int64

In [ ]:
note.to_csv('/content/drive/MyDrive/DLH_final_project_dataset/preprocessed_rolled_up_icd_codes_10000.csv')

In [ ]:
# add category
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
nltk.download('punkt')
class Preprocessing:
  def __init__(self, num_words, seq_len,category,target,binary=True):
    self.data = '/content/drive/MyDrive/DLH_final_project_dataset/preprocessed_reg_icd_codes_10000.csv'
    self.num_words = num_words
    self.seq_len = seq_len
    self.vocabulary = None
    self.x_tokenized = None
    self.x_padded = None
    self.x_raw = None
    self.y = None
    
    self.x_train = None
    self.x_test = None
    self.y_train = None
    self.y_test = None
    
    self.target=target
    self.category=category
  def load_data(self,binary):
    # Reads the raw csv file and split into
    # sentences (x) and target (y)
    df = pd.read_csv(self.data)
    #df.drop(['id','keyword','location'], axis=1, inplace=True)
    self.x_raw = df['TEXT'].values
    if (binary==False):
      #df['rolled_up_codes'] = df.loc[:, 'rolled_up_code_008':'rolled_up_code_V88'].values.tolist()
      #rolled_up_codes_list=['rolled_up_code_518','rolled_up_code_427','rolled_up_code_584','rolled_up_code_428','rolled_up_code_276',
                            #'rolled_up_code_401','rolled_up_code_285','rolled_up_code_414','rolled_up_code_272','rolled_up_code_038']
      #df['rolled_up_codes'] = df.loc[:, rolled_up_codes_list].values.tolist()
      #rolled_up_codes_list = notes.loc[:, notes.columns.str.startswith('rolled_up_code_')].columns.tolist()
      #df['rolled_up_codes'] = df.loc[:, rolled_up_codes_list[0]:rolled_up_codes_list[len(rolled_up_codes_list)-1]].values.tolist()
      #self.y = df['rolled_up_codes'].values
      reg_codes_list = notes.loc[:, notes.columns.str.startswith('reg_code_')].columns.tolist()
      df['reg_codes'] = df.loc[:, reg_codes_list[0]:reg_codes_list[len(reg_codes_list)-1]].values.tolist()
      self.y = df['reg_codes'].values
    else:
      self.y = df[self.target].values
   
    self.cat=df[self.category].values
  '''
  def clean_text(self):
    # Removes special symbols and just keep
    # words in lower or upper form
    self.x_raw = [x.lower() for x in self.x_raw]
    self.x_raw = [re.sub(r'[^A-Za-z]+', ' ', x) for x in self.x_raw]
  '''  
  def text_tokenization(self):
    # Tokenizes each sentence by implementing the nltk tool
    self.x_raw = [word_tokenize(x) for x in self.x_raw]
    
  def build_vocabulary(self):
    # Builds the vocabulary and keeps the "x" most frequent word
    self.vocabulary = dict()
    fdist = nltk.FreqDist()
    
    for sentence in self.x_raw:
      for word in sentence:
        fdist[word] += 1
        
    common_words = fdist.most_common(self.num_words)
    
    for idx, word in enumerate(common_words):
      self.vocabulary[word[0]] = (idx+1)
  
  def word_to_idx(self):
    # By using the dictionary (vocabulary), it is transformed
    # each token into its index based representatio
    self.x_tokenized = list() 
    
    for sentence in self.x_raw:
      temp_sentence = list()
      for word in sentence:
        if word in self.vocabulary.keys():
          temp_sentence.append(self.vocabulary[word])
      self.x_tokenized.append(temp_sentence)
      
  def padding_sentences(self):
    # Each sentence which does not fulfill the required le
    # it's padded with the index 0
    pad_idx = 0
    self.x_padded = list()
    
    for sentence in self.x_tokenized:
      while len(sentence) < self.seq_len:
        sentence.insert(len(sentence), pad_idx)
      self.x_padded.append(sentence)
 
    self.x_padded = np.array(self.x_padded)
    self.cat=self.cat.reshape(-1,1)
    self.x_padded=np.append(self.x_padded,self.cat,1)
  def split_data(self):
    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x_padded, self.y, test_size=0.25, random_state=42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def prepare_data(num_words, seq_len,category,target,binary=True):
  # Preprocessing pipeline
  pr = Preprocessing(num_words, seq_len,category,target,binary)
  pr.load_data(binary)
  #pr.clean_text()
  pr.text_tokenization()
  pr.build_vocabulary()
  pr.word_to_idx()
  pr.padding_sentences()
  pr.split_data()

  return {'x_train': pr.x_train, 'y_train': pr.y_train, 'x_test': pr.x_test, 'y_test': pr.y_test}

#data=prepare_data(45173, 2200,'category_Nursing/other','rolled_up_code_038',binary=True)
data=prepare_data(45173, 2200,'category_Respiratory','rolled_up_code_518',binary=False)

In [ ]:
len(data['y_test'][0])

1154

In [ ]:
# add category for Bot model for binary prediction
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
class BOT_cat(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,embedding_size=64,out_size=32):
      super(BOT_cat, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
  
      # Kernel sizes
      self.kernel_1 = 3
   
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Fully connected layer definition
      # Change based on situation:
      # binary classification
      self.fc = nn.Linear(65, 1)


   def forward(self, x):
      report=x[:,:-1]
      cat=torch.tensor(x[:,-1])
      cat=cat.reshape(-1,1)
      x = self.embedding(report)
 

      #x = [batch size, seq. length, hidden dim]
      
      x = F.avg_pool2d(x, (x.shape[1], 1)).squeeze(1) #averages word vectors across whole sequence length
      #x = [batch size, hidden dim]
      x=torch.cat((x,cat),1)
      x = self.fc(x)
      #x = [batch size, output dim]

      # Activation function is applied
      x = torch.sigmoid(x)
      
      return x.squeeze()

In [ ]:
# add category for Bot model for multiple prediction
class BOT_cat_mul(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,labels=1154,embedding_size=64,out_size=32):
      super(BOT_cat_mul, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
  
      # Kernel sizes
      self.kernel_1 = 3
   
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Fully connected layer definition
      # Change based on situation:
      # binary classification
      # self.fc = nn.Linear(64, 1)
      # multi-label classification
      self.fc = nn.Linear(65, labels)

   def forward(self, x):

      report=x[:,:-1]
      cat=torch.tensor(x[:,-1])

      cat=cat.reshape(-1,1)

      x = self.embedding(report)
      #x = [batch size, seq. length, hidden dim]
      x = F.avg_pool2d(x, (x.shape[1], 1)).squeeze(1) #averages word vectors across whole sequence length
      #x = [batch size, hidden dim]

      x=torch.cat((x,cat),1)
      x = self.fc(x)
      #x = [batch size, output dim]

      # Activation function is applied
      x = torch.sigmoid(x)
      
      return x.squeeze()

In [ ]:
# add category for CNN baseline model for binary prediction
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN_baseline_cat(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,embedding_size=64,out_size=32):
      super(CNN_baseline_cat, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
      
      # CNN parameters definition
      # Kernel sizes
      self.kernel_1 = 3

      
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Convolution layers definition
      self.conv_1 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_1)
      
      # Max pooling layers definition
      self.pool_1 = nn.MaxPool1d(self.kernel_1)
      
      # Fully connected layer definition
      self.fc = nn.Linear(641, 1)
 
   def forward(self, x):
      
      # Sequence of tokes is filterd through an embedding layer
      report=x[:,:-1]
      cat=torch.tensor(x[:,-1])

      cat=cat.reshape(-1,1)


      x = self.embedding(report)

      #x=torch.cat((x,cat),1)
      # Convolution layer 1 is applied
      x1 = self.conv_1(x)
      x1 = torch.relu(x1)
      x1 = self.pool_1(x1)

      # The output of each convolutional layer is concatenated into a unique vector
      x1=torch.flatten(x1, 1)

      x1=torch.cat((x1,cat),1)
      # The "flattened" vector is passed through a fully connected layer
      out = self.fc(x1)
      # Activation function is applied
      out = torch.sigmoid(out)
      
      return out.squeeze()

In [ ]:
class CNN_baseline_cat_mul(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,labels=1154,embedding_size=64,out_size=32):
      super(CNN_baseline_cat_mul, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
      
      # CNN parameters definition
      # Kernel sizes
      self.kernel_1 = 3

      
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Convolution layers definition
      self.conv_1 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_1)
      
      # Max pooling layers definition
      self.pool_1 = nn.MaxPool1d(self.kernel_1)
      
      # Fully connected layer definition
      self.fc = nn.Linear(641, labels)
 
   def forward(self, x):
      
      # Sequence of tokes is filterd through an embedding layer
      
      report=x[:,:-1]
      cat=torch.tensor(x[:,-1])
      cat=cat.reshape(-1,1)
      
      x = self.embedding(report)

      #x=torch.cat((x,cat),1)
      # Convolution layer 1 is applied
      x1 = self.conv_1(x)
      x1 = torch.relu(x1)
      x1 = self.pool_1(x1)

      
      # The output of each convolutional layer is concatenated into a unique vector
      x1=torch.flatten(x1, 1)
      x1=torch.cat((x1,cat),1)
      # The "flattened" vector is passed through a fully connected layer
      out = self.fc(x1)
      # Activation function is applied
      out = torch.sigmoid(out)
      
      return out.squeeze()

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader

class DatasetMaper(Dataset):

	def __init__(self, x, y):
		self.x = x
		self.y = y
		
	def __len__(self):
		return len(self.x)
		
	def __getitem__(self, idx):
		return self.x[idx], self.y[idx]

In [ ]:
class Run:
    '''Training, evaluation and metrics calculation'''

    @staticmethod
    def train(model, data, multi=10,binary=True):
        
        # Initialize dataset maper
        train = DatasetMaper(data['x_train'], data['y_train'])
        test = DatasetMaper(data['x_test'], data['y_test'])
        
        # Initialize loaders
        loader_train = DataLoader(train, batch_size=32)
        loader_test = DataLoader(test, batch_size=32)
        
        # Define optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Starts training phase
        for epoch in range(15):
            # Set model in training model
            model.train()
            predictions = []
            # Starts batch training
            for x_batch, y_batch in loader_train:
                if (binary==False):
                  y_batch = torch.stack(y_batch).T
                y_batch = y_batch.type(torch.FloatTensor)
                
                # Feed the model
                y_pred = model(x_batch)

                # Loss calculation
                loss = F.binary_cross_entropy(y_pred, y_batch)
                
                # Clean gradientes
                optimizer.zero_grad()
                
                # Gradients calculation
                loss.backward()
                
                # Gradients update
                optimizer.step()
                
                # Save predictions
                predictions += list(y_pred.detach().numpy())
            
            # Evaluation phase
            test_predictions = Run.evaluation(model, loader_test)
            if (binary==True):
              predictions=[1 if pred>0.5 else 0 for pred in predictions]

              test_predictions=[1 if pred>0.5 else 0 for pred in test_predictions]

              train_precision,train_recall,train_f1,_ = precision_recall_fscore_support(predictions,data['y_train'],average='binary')
              test_precision,test_recall,test_f1,_ = precision_recall_fscore_support(test_predictions,data['y_test'],average='binary')
              test_accuracy=accuracy_score(data['y_test'],test_predictions)
              print("Epoch: %d, loss: %.5f, Train precision: %.5f,Train recall: %.5f,Train f1: %.5f, Test accuracy: %.5f,Test precision: %.5f, Test recall: %.5f,Test f1: %.5f" % (epoch+1, loss.item(), train_precision,train_recall,train_f1,test_accuracy,test_precision,test_recall,test_f1))
            else:
              for i in range(len(predictions)):
                predictions[i]=[1 if pred>0.5 else 0 for pred in predictions[i]]  

              for i in range(len(test_predictions)):
                test_predictions[i]=[1 if pred>0.5 else 0 for pred in test_predictions[i]] 
              
              multi_label_predictions = np.array(predictions)
              print(multi_label_predictions.shape)
              y_train=np.empty([7500,multi])
              for i in range(len(data['y_train'])):
                y_train[i][:]=data['y_train'][i]

              train_precision_all = np.empty([multi])
              train_recall_all = np.empty([multi])
              train_f1_all = np.empty([multi])
              for i in range(len(predictions[0])):
                  train_precision_all[i],train_recall_all[i],train_f1_all[i],_ = precision_recall_fscore_support(multi_label_predictions[:][i],y_train[:][i],average='binary')
              train_precision = train_precision_all.mean()
              train_recall = train_recall_all.mean()
              train_f1 = train_f1_all.mean()

              multi_label_test_predictions = np.array(test_predictions)
              print(multi_label_test_predictions.shape)
              y_test=np.empty([2500,multi])
              for i in range(len(data['y_test'])):
                y_test[i][:]=data['y_test'][i]

              test_precision_all = np.empty([multi])
              test_recall_all = np.empty([multi])
              test_f1_all = np.empty([multi])
              for i in range(len(test_predictions[0])):
                  test_precision_all[i],test_recall_all[i],test_f1_all[i],_ = precision_recall_fscore_support(multi_label_test_predictions[:][i],y_test[:][i],average='binary')
              test_precision = test_precision_all.mean()
              test_recall = test_recall_all.mean()
              test_f1 = test_f1_all.mean()
              print("Epoch: %d, loss: %.5f, Train precision: %.5f,Train recall: %.5f,Train f1: %.5f, Test precision: %.5f, Test recall: %.5f,Test f1: %.5f" % (epoch+1, loss.item(), train_precision,train_recall,train_f1,test_precision,test_recall,test_f1))
            
    @staticmethod
    def evaluation(model, loader_test):
        
        # Set the model in evaluation mode
        model.eval()
        predictions = []
        
        # Starst evaluation phase
        with torch.no_grad():
            for x_batch, y_batch in loader_test:
                y_pred = model(x_batch)
                predictions += list(y_pred.detach().numpy())
        return predictions

In [ ]:
model_bot=BOT_cat()
Run().train(model_bot,data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.

Epoch: 1, loss: 0.63923, Train precision: 0.00000,Train recall: 0.00000,Train f1: 0.00000, Test accuracy: 0.75480,Test precision: 0.00000, Test recall: 0.00000,Test f1: 0.00000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 2, loss: 0.63996, Train precision: 0.00000,Train recall: 0.00000,Train f1: 0.00000, Test accuracy: 0.75480,Test precision: 0.00000, Test recall: 0.00000,Test f1: 0.00000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

Epoch: 13, loss: 0.17495, Train precision: 0.53626,Train recall: 0.98189,Train f1: 0.69367, Test accuracy: 0.86480,Test precision: 0.47471, Test recall: 0.94788,Test f1: 0.63261


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 14, loss: 0.15732, Train precision: 0.55604,Train recall: 0.98062,Train f1: 0.70968, Test accuracy: 0.86600,Test precision: 0.48450, Test recall: 0.93987,Test f1: 0.63940


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 15, loss: 0.14250, Train precision: 0.58022,Train recall: 0.97687,Train f1: 0.72802, Test accuracy: 0.87000,Test precision: 0.50245, Test recall: 0.93902,Test f1: 0.65462


In [ ]:
model_cnn=CNN_baseline_cat()
Run().train(model_cnn,data)

In [ ]:
model_bot_mul=BOT_cat_mul()
Run().train(model_bot_mul,data,multi=1154,binary=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)
(2500, 1154)
Epoch: 1, loss: 0.39562, Train precision: 0.33129,Train recall: 0.01775,Train f1: 0.03316, Test precision: 0.07301, Test recall: 0.59792,Test f1: 0.12722


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)
(2500, 1154)
Epoch: 2, loss: 0.24137, Train precision: 0.07645,Train recall: 0.62926,Train f1: 0.13340, Test precision: 0.07108, Test recall: 0.60240,Test f1: 0.12428


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

(7500, 1154)
(2500, 1154)
Epoch: 13, loss: 0.10924, Train precision: 0.07950,Train recall: 0.63104,Train f1: 0.13814, Test precision: 0.07401, Test recall: 0.60226,Test f1: 0.12874


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)
(2500, 1154)
Epoch: 14, loss: 0.10633, Train precision: 0.07981,Train recall: 0.62955,Train f1: 0.13855, Test precision: 0.07434, Test recall: 0.60334,Test f1: 0.12925


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)
(2500, 1154)
Epoch: 15, loss: 0.10369, Train precision: 0.08025,Train recall: 0.62971,Train f1: 0.13914, Test precision: 0.07468, Test recall: 0.60354,Test f1: 0.12974


In [ ]:
model_cnn_mul=CNN_baseline_cat_mul()
Run().train(model_cnn_mul,data,multi=1154,binary=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and 

(2500, 1154)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and 

...

(7500, 1154)
(2500, 1154)
Epoch: 13, loss: 0.00290, Train precision: 0.81410,Train recall: 0.91646,Train f1: 0.85194, Test precision: 0.40171, Test recall: 0.53497,Test f1: 0.43752


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)
(2500, 1154)
Epoch: 14, loss: 0.00256, Train precision: 0.83596,Train recall: 0.92728,Train f1: 0.87055, Test precision: 0.40708, Test recall: 0.52942,Test f1: 0.43956


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(7500, 1154)
(2500, 1154)
Epoch: 15, loss: 0.00228, Train precision: 0.85438,Train recall: 0.93566,Train f1: 0.88559, Test precision: 0.40700, Test recall: 0.52206,Test f1: 0.43742
